In [1]:
import numpy as np
import scipy as sp
import pandas as pd

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor

In [108]:
# reading data
data = pd.read_csv("year_prediction.csv")
data = data.rename(index=str, columns={"label":"year"})

In [111]:
# separate input attributes and output into different dataframes
X = data.iloc[:,1:]
Y = data.iloc[:,0]

# Train set
X_train = X.iloc[0:463715,:]
y_train = Y.iloc[0:463715]

# Validation set
X_test = X.iloc[463715:,:]
y_test = Y.iloc[463715:]

In [112]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

# if we want to standarize the training data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [256]:
num_of_rep = 20

linearReg = Ridge(alpha=46371.5)

train_res = np.zeros(num_of_rep)
test_res = np.zeros(num_of_rep)

for t in range(num_of_rep):
    linearReg.fit(X_train, y_train)
    pred_train = linearReg.predict(X_train)
    total_cost_train = np.sum((pred_train - y_train) ** 2) / 463715 + 0.1 * np.sum((linearReg.coef_) ** 2)
    print(total_cost_train)

    pred_test = linearReg.predict(X_test)
    total_cost_test = np.sum((pred_test - y_test) ** 2) / 51630
    print(total_cost_test)
    
    train_res[t] = total_cost_train
    test_res[t] = total_cost_test
    
name_str1 = "./res/ridge/ridge-centralized-train.npy"
name_str2 = "./res/ridge/ridge-centralized-test.npy" 
np.save(name_str1, train_res)
np.save(name_str2, test_res)

91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518
91.34207242375507
90.44726168835518


In [182]:
def gd(X, y, lamb = 0.1, eta = 0.1, T = 1000, gamma = 0.99):
    num_data, d = X.shape
    y = y.reshape(-1,1)
    m = np.zeros((d,1))
    res = np.zeros((d,1))
    for t in range(T):
        #print(res)
        if t % 10000 == 0:
            print(t)
        grad = X.T @ X @ res / num_data - X.T @ y / num_data + lamb * res
        m = gamma * m + (1-gamma) * grad
        res = res - eta * m
    return res

In [150]:
res = gd(X_train_std, y_train)

pred_train = (X_train_std @ res).reshape(-1)
total_cost_train = np.sum((pred_train - y_train) ** 2) / 463715 + np.sum((res.reshape(-1)) ** 2)
print(total_cost_train)

pred_test = (X_test_std @ res).reshape(-1)
total_cost_test = np.sum((pred_test - y_test) ** 2) / 51630
print(total_cost_test)

0
100
200
300
400
500
600
700
800
900
3993671.583046587
3993751.2675514827


In [221]:
# construct coreset
def coreset_lr(m, D, y):
    # m is the number of samples, X is the data
    num_of_data, _ = D.shape
    D = np.hstack((D,y.reshape(-1,1)))
    #s = np.sum(D ** 2, axis=1) + 1 / num_of_data
    D1 = D[:,:30]
    D2 = D[:,30:60]
    D3 = D[:,60:]
    q1, _ = np.linalg.qr(D1)
    q2, _ = np.linalg.qr(D2)
    q3, _ = np.linalg.qr(D2)
    Q = np.hstack((q1,q2,q3))
    
    s = np.sum(Q ** 2, axis=1) + 1/num_of_data
    #D = np.hstack((D, y.to_numpy().reshape(-1,1)))
    D = np.hstack((D, (1/s).reshape(-1,1)))
    D_df = pd.DataFrame(D)
    C = D_df.sample(n=m, replace=False, weights=s)
    C = C.to_numpy()
    data = C[:,:-2]
    label = C[:,-2]
    weight = C[:,-1]
    weight = weight / np.sum(weight) * m * 10
    return data, label, weight

def uniform_lr(m,D,y):
    D = np.hstack((D, y.reshape(-1,1)))
    #D = np.hstack((D, y.to_numpy().reshape(-1,1)))
    D_df = pd.DataFrame(D)
    C = D_df.sample(n=m, replace=False)
    C = C.to_numpy()
    data = C[:,:-1]
    label = C[:,-1]
    return data, label

In [250]:
num_of_rep = 20
size_list = [1000,2000,3000,4000,5000,6000]

linearReg = Ridge(alpha=46371.5)

for size in size_list:
    train_res = np.zeros(num_of_rep)
    test_res = np.zeros(num_of_rep)
    for t in range(num_of_rep):
        coreset_X, coreset_y, weight = coreset_lr(size,X_train,y_train)
        linearReg.fit(coreset_X, coreset_y, weight)

        #coreset_X, coreset_y, _ = coreset_lr(5000,X_train,y_train)
        #linearReg.fit(coreset_X, coreset_y)

        pred_train = linearReg.predict(X_train)
        total_cost_train = np.sum((pred_train - y_train) ** 2) / 463715 + 0.1 * np.sum((linearReg.coef_) ** 2)

        pred_test = linearReg.predict(X_test)
        total_cost_test = np.sum((pred_test - y_test) ** 2) / 51630
    
        train_res[t] = total_cost_train
        test_res[t] = total_cost_test
    
    name_str1 = "./res/ridge/ridge-centralized-c"+str(size)+"s-train.npy"
    name_str2 = "./res/ridge/ridge-centralized-c"+str(size)+"s-test.npy" 
    np.save(name_str1, train_res)
    np.save(name_str2, test_res)

In [257]:
num_of_rep = 20
size_list = [1200]

linearReg = Ridge(alpha=46371.5)

for size in size_list:
    train_res = np.zeros(num_of_rep)
    test_res = np.zeros(num_of_rep)
    for t in range(num_of_rep):
        uniform_X, uniform_y = uniform_lr(size,X_train,y_train)
        linearReg.fit(uniform_X, uniform_y)

        pred_train = linearReg.predict(X_train)
        total_cost_train = np.sum((pred_train - y_train) ** 2) / 463715 + 0.1 * np.sum((linearReg.coef_) ** 2)

        pred_test = linearReg.predict(X_test)
        total_cost_test = np.sum((pred_test - y_test) ** 2) / 51630
        
        train_res[t] = total_cost_train
        test_res[t] = total_cost_test
    
    name_str1 = "./res/ridge/ridge-centralized-u"+str(size)+"s-train.npy"
    name_str2 = "./res/ridge/ridge-centralized-u"+str(size)+"s-test.npy" 
    np.save(name_str1, train_res)
    np.save(name_str2, test_res)

In [258]:
num_of_rep = 20
size_list = [1000]

new_lr = Ridge(alpha=46371.5,solver='saga',max_iter=10000,tol=1e-4)

for size in size_list:
    train_res = np.zeros(num_of_rep)
    test_res = np.zeros(num_of_rep)
    iter_res = np.zeros(num_of_rep)
    for t in range(num_of_rep):
        coreset_X, coreset_y, weight = coreset_lr(size,X_train,y_train)

        new_lr.fit(coreset_X, coreset_y, weight)

        pred_train = new_lr.predict(X_train)
        total_cost_train = np.sum((pred_train - y_train) ** 2) / 463715 + 0.1 * np.sum((linearReg.coef_) ** 2)

        pred_test = new_lr.predict(X_test)
        total_cost_test = np.sum((pred_test - y_test) ** 2) / 51630
        
        train_res[t] = total_cost_train
        test_res[t] = total_cost_test
        iter_res[t] = new_lr.n_iter_[0]

    name_str1 = "./res/ridge/ridge-saga-c"+str(size)+"s-train.npy"
    name_str2 = "./res/ridge/ridge-saga-c"+str(size)+"s-test.npy" 
    name_str3 = "./res/ridge/ridge-saga-c"+str(size)+"s-iter.npy" 
    np.save(name_str1, train_res)
    np.save(name_str2, test_res)
    np.save(name_str3, iter_res)

In [261]:
num_of_rep = 20
size_list = [1000,2000,3000,4000,5000,6000]

for size in size_list:
    train_res = np.zeros(num_of_rep)
    test_res = np.zeros(num_of_rep)
    iter_res = np.zeros(num_of_rep)
    for t in range(num_of_rep):

        uniform_X, uniform_y = uniform_lr(size,X_train,y_train)

        new_lr = Ridge(alpha=46371.5,solver='saga',max_iter=10000,tol=8e-5)
        new_lr.fit(uniform_X, uniform_y)

        pred_train = new_lr.predict(X_train)
        total_cost_train = np.sum((pred_train - y_train) ** 2) / 463715 + 0.1 * np.sum((linearReg.coef_) ** 2)

        pred_test = new_lr.predict(X_test)
        total_cost_test = np.sum((pred_test - y_test) ** 2) / 51630

        train_res[t] = total_cost_train
        test_res[t] = total_cost_test
        iter_res[t] = new_lr.n_iter_[0]

    name_str1 = "./res/ridge/ridge-saga-u"+str(size)+"s-train.npy"
    name_str2 = "./res/ridge/ridge-saga-u"+str(size)+"s-test.npy" 
    name_str3 = "./res/ridge/ridge-saga-u"+str(size)+"s-iter.npy" 
    np.save(name_str1, train_res)
    np.save(name_str2, test_res)
    np.save(name_str3, iter_res)

C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn

In [286]:
a = 1
for i in range(1000):
    a *= (500000-i) / 500000
print(a)

0.3680022108329327


In [10]:
# reading data
data = pd.read_csv("kc_house_data.csv")

data.describe()

y = data.iloc[:,2].to_numpy()
X = data.iloc[:,3:].to_numpy()

num_of_data, _ = X.shape
print(X.shape)

print(y[:10])

(21613, 18)
[ 221900.  538000.  180000.  604000.  510000. 1225000.  257500.  291850.
  229500.  323000.]


In [15]:
num_of_rep = 20

linearReg = Ridge(alpha=0)

train_res = np.zeros(num_of_rep)

for t in range(num_of_rep):
    linearReg.fit(X, y)
    pred_train = linearReg.predict(X)
    total_cost_train = np.sum((pred_train - y) ** 2) / 21613
    print(np.sqrt(total_cost_train))
    
    train_res[t] = total_cost_train
    print(linearReg.coef_)
    
#name_str1 = "./res/ridge/ridge-centralized-train.npy"
#name_str2 = "./res/ridge/ridge-centralized-test.npy" 
#np.save(name_str1, train_res)
#np.save(name_str2, test_res)

201163.90222277155
[-3.57665414e+04  4.11442785e+04  2.11485657e+02  1.28597869e-01
  6.68955012e+03  5.82960458e+05  5.28709424e+04  2.63856491e+04
  9.58904452e+04 -3.02575684e+01 -6.13851528e+01 -2.62022321e+03
  1.98125837e+01 -5.82419866e+02  6.02748226e+05 -2.14729828e+05
  2.16814005e+01 -3.82641850e-01]
201163.90222277155
[-3.57665414e+04  4.11442785e+04  2.11485657e+02  1.28597869e-01
  6.68955012e+03  5.82960458e+05  5.28709424e+04  2.63856491e+04
  9.58904452e+04 -3.02575684e+01 -6.13851528e+01 -2.62022321e+03
  1.98125837e+01 -5.82419866e+02  6.02748226e+05 -2.14729828e+05
  2.16814005e+01 -3.82641850e-01]
201163.90222277155
[-3.57665414e+04  4.11442785e+04  2.11485657e+02  1.28597869e-01
  6.68955012e+03  5.82960458e+05  5.28709424e+04  2.63856491e+04
  9.58904452e+04 -3.02575684e+01 -6.13851528e+01 -2.62022321e+03
  1.98125837e+01 -5.82419866e+02  6.02748226e+05 -2.14729828e+05
  2.16814005e+01 -3.82641850e-01]
201163.90222277155
[-3.57665414e+04  4.11442785e+04  2.114856

C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.011e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.011e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.011e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.011e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages

In [8]:
# construct coreset
def coreset_lr(m, D, y):
    # m is the number of samples, X is the data
    num_of_data, _ = D.shape
    D = np.hstack((D,y.reshape(-1,1)))
    #s = np.sum(D ** 2, axis=1) + 1 / num_of_data
    D1 = D[:,:10]
    D2 = D[:,10:]
    q1, _ = np.linalg.qr(D1)
    q2, _ = np.linalg.qr(D2)
    Q = np.hstack((q1,q2))
    
    s = np.sum(Q ** 2, axis=1) + 1/num_of_data
    #D = np.hstack((D, y.to_numpy().reshape(-1,1)))
    D = np.hstack((D, (1/s).reshape(-1,1)))
    D_df = pd.DataFrame(D)
    C = D_df.sample(n=m, replace=False, weights=s)
    C = C.to_numpy()
    data = C[:,:-2]
    label = C[:,-2]
    weight = C[:,-1]
    weight = weight / np.sum(weight) * m * 10
    return data, label, weight

def uniform_lr(m,D,y):
    D = np.hstack((D, y.reshape(-1,1)))
    #D = np.hstack((D, y.to_numpy().reshape(-1,1)))
    D_df = pd.DataFrame(D)
    C = D_df.sample(n=m, replace=False)
    C = C.to_numpy()
    data = C[:,:-1]
    label = C[:,-1]
    return data, label

In [20]:
num_of_rep = 20
size_list = [100,200,300,400,500,600]

linearReg = Ridge(alpha=0)

for size in size_list:
    print("coreset size %d" % size)
    train_res = np.zeros(num_of_rep)
    test_res = np.zeros(num_of_rep)
    for t in range(num_of_rep):
        coreset_X, coreset_y, weight = coreset_lr(size,X,y)
        linearReg.fit(coreset_X, coreset_y, weight)

        #coreset_X, coreset_y, _ = coreset_lr(5000,X_train,y_train)
        #linearReg.fit(coreset_X, coreset_y)

        pred_train = linearReg.predict(X)
        total_cost_train = np.sum((pred_train - y) ** 2) / 21613
        print(np.sqrt(total_cost_train))
    
        train_res[t] = total_cost_train
    
    #name_str1 = "./res/ridge/ridge-centralized-c"+str(size)+"s-train.npy"
    #name_str2 = "./res/ridge/ridge-centralized-c"+str(size)+"s-test.npy" 
    #np.save(name_str1, train_res)
    #np.save(name_str2, test_res)

coreset size 100
220637.99474910603
214352.29963899092
220942.05013600527
231170.42822098936
218165.46038470566
214025.7479015629
211586.96031055987
230938.49981882918
228099.20159674852


C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.75308e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.03212e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.50799e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.012e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-pa

852446.6974063414
211854.1753627322
259561.48079279132
214345.01684348183
216096.51192824385
216991.25398877106
236316.11311314622
216609.43110978793
217997.3577445498
212201.11236173278
288209.1974440927
coreset size 200


C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.15107e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.71093e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=7.81128e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=6.2275e-21): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-p

208992.51315195367
261117.8998724629
215214.6303896498
207445.14537342978
205328.0506912772
205897.68139641942
214819.57026473506
265917.45877240226
208164.34032587704
213447.6023150168
267997.1088598232


...

C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.36838e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.21969e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.13474e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=6.65131e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-

202930.38070683024
203217.44227937228
202601.7098287859
204400.82662099376
206999.62068920385
202841.58210067806
205397.84573435248
204109.2092990753
1271920.188007641
203601.75928982793
coreset size 600
202721.6099697737
203933.96437947312
202947.2670944284
204621.5080166854


C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.19122e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=5.90951e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.83549e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=6.50783e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-

203944.68837992515
205160.0899354579
207082.5871430924
203692.43432556733
202600.070657696
202080.9145720824
205529.0068689157
204065.70798688236
209714.30445242726
202813.4329188228
202811.10981279396
205538.74854716804
204442.13804203892
203055.33783648626
204811.0411432676
274622.9715308454


C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=5.08718e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.33889e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.4974e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.05376e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-p

In [23]:
num_of_rep = 20
size_list = [100,200,300,400,500,600]

linearReg = Ridge(alpha=0)

for size in size_list:
    print("uniform size %d" % size)
    train_res = np.zeros(num_of_rep)
    test_res = np.zeros(num_of_rep)
    for t in range(num_of_rep):
        uniform_X, uniform_y = uniform_lr(size,X,y)
        linearReg.fit(uniform_X, uniform_y)

        pred_train = linearReg.predict(X)
        total_cost_train = np.sum((pred_train - y) ** 2) / 21613
        print(np.sqrt(total_cost_train))
        
        train_res[t] = total_cost_train
    
    #name_str1 = "./res/ridge/ridge-centralized-u"+str(size)+"s-train.npy"
    #name_str2 = "./res/ridge/ridge-centralized-u"+str(size)+"s-test.npy" 
    #np.save(name_str1, train_res)
    #np.save(name_str2, test_res)

uniform size 100
1.1467923607867667e+17
235347.04333491818
4.7429318935817434e+17
3.819095558183439e+16
216910.3938035999
257022.04306108752
499952.0046487101
6.835985190251619e+16
1.7035402994019606e+16
7482539873417389.0
4.615553906586046e+16
2.1646320837572643e+17
229497.74741459906
260067.43224950196
228077.18440116313
3.840858320627686e+16
220133.45384094995
1.3606474397251174e+17
252861.31145938553
3.1735546830472224e+16
uniform size 200
230227.89983717736
2.431153154681409e+17
8.672973416435275e+16
407909.90646526683
253113.41878085062
258528.0850984053
228994.80754280434
234920.89209565098
217692.26797536883
214898.70760604704
209759.9763988243
227731.700310889
3.564217272939315e+16
228208.74226126185
233274.61583681882
299303.60029192397
238923.8455492833
214483.7868592951
210129.77736161402
325848.74445063266
uniform size 300
211383.7166056081
207682.92875136586
322198.1126314549
214366.06364708656
213123.15767368264
256383.54442642257
216177.1117262468
611648.5730852516

C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.38682e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.25176e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.84375e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=5.37407e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-


210863.35125604933
291776.871536307
1.6485644614342824e+16
210219.4457865605
220073.0709677633
258934.57539131804
214243.9152765675
207666.45246771353
264259.157662853
290068.38351422746
208589.86871224878
2.531506308617894e+16
uniform size 400
258380.37250947364
208853.04280592472
225741.69021386295
248853.3007318634
217224.64951997
216724.142250274
232083.9257154845
217250.28639670264
205804.59255199952
207909.0627269818
216398.42399797676
204651.66260364614
215461.86106629195
220142.2211848624
356042.8267266693
207682.40477481735
211184.01745152922
216353.5054106522
259663.64960312418
208975.66281725952
uniform size 500
209805.70789959523
223356.48590641315
224066.63939775852
208594.5476315648
476215.0107557676
8139060.875409994
210506.55863289
209166.3617185218
234146.6618143504
213665.14040493296
225607.67637546896
215757.53485663698
210429.29129432573
211766.16007344658


C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.23568e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=8.35913e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=5.81514e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.24149e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-

211249.31588071262
214327.44075322422
209041.591434925
267116.9314170397
205804.86839637609
431856.7788381895
uniform size 600
214213.77306249834
236670.77672768687
207422.72628668512
447899.54460285703
210673.11302495224
208249.797258985
288278.2814858876
205894.3812464596
249705.49459550442
215037.12779016426
209273.39327721583
209063.02698286946
312460.23221584444
233504.53107550665
204461.11234272626
209563.0044415892
206024.90507904845
223389.82757679137
269987.8486617362
204648.16770216692


C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.27973e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=7.50541e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.95327e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.42656e-19): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\thoma\anaconda3\envs\py38\lib\site-